# Using FABRIC Xilinx FPGA U280 with P4

Your compute nodes can include FPGAs. These devices are made available as FABRIC components and can be added to your nodes like any other component. Your project must have Component.FPGA permission tag in order to be able to provision them. 

This example notebook will demonstrate how to reserve and use a single Xilinx FPGA device on FABRIC and use it in conjunction with ConnectX-6 cards. It creates a slice with two nodes - one with FPGA and another with a ConnectX-6 card connected via L2Bridge L2 services. A sample P4 application is loaded using ESnet workflow and a traffic is exchanged between the FPGA and the ConnectX-6 card.


## Setup the Experiment

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Credential Manager,beta-2.fabric-testbed.net
Orchestrator,beta-7.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,b9847fa1-13ef-49f9-9e07-ae6ad06cda3f
Bastion Username,ibaldin_0000241998
Bastion Private Key File,/home/fabric/work/fabric_config/KNIT6TutorialKey
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Select a site with available FPGA

The cells below help you create a slice that contains a single node with an attached FPGA. 

In [2]:
FPGA_CHOICE='FPGA_Xilinx_U280'
SMART_NIC_CHOICE='NIC_ConnectX_6'

# don't edit - convert from FPGA type to a resource column name
# to use in filter lambda function below
choice_to_column = {
    "FPGA_Xilinx_U280": "fpga_u280_available",
}

column_name = choice_to_column.get(FPGA_CHOICE, "Unknown")
print(f'{column_name=}')

column_name='fpga_u280_available'


Give the slice and the node in it meaningful names.

In [3]:
# name the slice and the node 
slice_name=f'My Simple FPGA with P4 Slice with {FPGA_CHOICE}'
storage_name='xilinx-tools'
mount_point = 'xilinx-tools'
fpga_node_name='fpga-node'
cx6_node_name='cx-6-node'
l2bridge1_name='l2bridge1'
l2bridge2_name='l2bridge2'

print(f'Will create slice "{slice_name}" with node "{fpga_node_name}" and node "{cx6_node_name}"')

Will create slice "My Simple FPGA with P4 Slice with FPGA_Xilinx_U280" with node "fpga-node" and node "cx-6-node"


Use a lambda filter to figure out which site the node will go to.

In [4]:
import random

# you can limit to one of the sites on this list (or use None)
#allowed_sites = ['INDI']
allowed_sites = None

fpga_sites_df = fablib.list_sites(output='pandas', quiet=True, filter_function=lambda x: x[column_name] > 0, force_refresh=True)
# note that list_sites with 'pandas' doesn't actually return a dataframe like doc sez, it returns a Styler 
# based on the dataframe
if fpga_sites_df:
    fpga_sites = fpga_sites_df.data['Name'].values.tolist()
else:
    fpga_sites = []
print(f'All sites with FPGA available: {fpga_sites}')

if len(fpga_sites)==0:
    print('Warning - no sites with available FPGAs found')
else:
    if allowed_sites and len(allowed_sites) > 0:
        fpga_sites = list(set(fpga_sites) & set(allowed_sites))
    if len(fpga_sites) == 0:
        print('Unable to find sites with available FPGAs')
    else:
        print('Selecting a site at random ' + f'among {allowed_sites}' if allowed_sites else '')

        site = random.choice(fpga_sites)
        print(f'Preparing to create slice "{slice_name}" with nodes {fpga_node_name} and {cx6_node_name} in site {site}')
        
# final site override if needed
site = 'RENC'

All sites with FPGA available: ['RENC']

Preparing to create slice "My Simple FPGA with P4 Slice with FPGA_Xilinx_U280" with nodes fpga-node and cx-6-node in site RENC


## Create a slice with a node with FPGA at desired site

This slice has two VMs - one with the FPGA and the other with a ConnectX-6 card - we will want to pass traffic between them.

In [5]:
# Create Slice. Note that by default submit() call will poll for 360 seconds every 10-20 seconds
# waiting for slice to come up. Normal expected time is around 2 minutes. 
slice = fablib.new_slice(name=slice_name)
image = 'docker_ubuntu_20'

# Add node with a 100G drive and 8 of CPU cores using Ubuntu 20 image
node1 = slice.add_node(name=fpga_node_name, site=site, cores=8, ram=8, disk=100, image=image)
node1.add_storage(name=storage_name)
fpga_comp = node1.add_component(model=FPGA_CHOICE, name='fpga1')
fpga_p1 = fpga_comp.get_interfaces()[0]
fpga_p2 = fpga_comp.get_interfaces()[1]
# be sure to add FABNetv4 so we can communicate with the slice that has the tools
#node1.add_fabnet()

# Add another node with ConnectX-6 cards of similar dimensions
node2 = slice.add_node(name=cx6_node_name, site=site, cores=8, disk=100, image=image)
#node2 = slice.add_node(name=cx6_node_name, site=site, cores=8, disk=100)
cx6_comp = node2.add_component(model=SMART_NIC_CHOICE, name='nic1')
cx6_p1 = cx6_comp.get_interfaces()[0]
cx6_p2 = cx6_comp.get_interfaces()[1]
#node2.add_fabnet()

# Use L2Bridge network services to connect the smart NIC and the FPGA ports
net1 = slice.add_l2network(name=l2bridge1_name, interfaces=[fpga_p1, cx6_p1], type='L2Bridge')
net2 = slice.add_l2network(name=l2bridge2_name, interfaces=[fpga_p2, cx6_p2], type='L2Bridge')

# Submit Slice Request
slice.submit();


Retry: 11, Time: 283 sec


ID,e118f4ce-537d-4644-849c-5690137cb610
Name,My Simple FPGA with P4 Slice with FPGA_Xilinx_U280
Lease Expiration (UTC),2023-09-21 21:28:21 +0000
Lease Start (UTC),2023-09-20 21:28:21 +0000
Project ID,b9847fa1-13ef-49f9-9e07-ae6ad06cda3f
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
b6313d6b-3a6c-4c05-a50a-d4f74fc16ece,cx-6-node,8,8,100,docker_ubuntu_20,qcow2,renc-w2.fabric-testbed.net,RENC,ubuntu,152.54.15.46,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@152.54.15.46,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
123245b7-570e-44ef-8bb4-35b1ff383d11,fpga-node,8,8,100,docker_ubuntu_20,qcow2,renc-w2.fabric-testbed.net,RENC,ubuntu,152.54.15.45,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@152.54.15.45,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
e1b82045-687a-44d5-837f-bac092950f2f,l2bridge1,L2,L2Bridge,RENC,None,None,Active,
4df55c1e-5c77-4999-8730-9c7007f8272a,l2bridge2,L2,L2Bridge,RENC,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
fpga-node-fpga1-p1,p1,fpga-node,l2bridge1,100,config,,None,None,None,None,2
fpga-node-fpga1-p2,p2,fpga-node,l2bridge2,100,config,,None,None,None,None,2
cx-6-node-nic1-p2,p2,cx-6-node,l2bridge2,100,config,,04:3F:72:B7:15:75,enp8s0,enp8s0,None,1
cx-6-node-nic1-p1,p1,cx-6-node,l2bridge1,100,config,,04:3F:72:B7:15:74,enp7s0,enp7s0,None,1



Time to print interfaces 285 seconds


# Setup IOMMU and Hugepages
For DPDK to function properly we need to setup hugepages and IOMMU on the VM

In [6]:
slice = fablib.get_slice(name=slice_name)
node1 = slice.get_node(name=fpga_node_name)

commands = list()
#commands.append("sudo sed -i 's/GRUB_CMDLINE_LINUX=\"\\(.*\\)\"/GRUB_CMDLINE_LINUX=\"\\1 amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=8\"/' /etc/default/grub")
commands.append("sudo sed -i 's/GRUB_CMDLINE_LINUX=\"\"/GRUB_CMDLINE_LINUX=\"amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=8\"/' /etc/default/grub")
commands.append("sudo grub-mkconfig -o /boot/grub/grub.cfg")
commands.append("sudo update-grub")

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node1.execute(command)
    
print('Done')

Executing sudo sed -i 's/GRUB_CMDLINE_LINUX=""/GRUB_CMDLINE_LINUX="amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=8"/' /etc/default/grub
Executing sudo grub-mkconfig -o /boot/grub/grub.cfg
 Sourcing file `/etc/default/grub'
Sourcing file `/etc/default/grub.d/50-cloudimg-settings.cfg'
Sourcing file `/etc/default/grub.d/init-select.cfg'
Generating grub configuration file ...
Found linux image: /boot/vmlinuz-5.4.0-155-generic
Found initrd image: /boot/initrd.img-5.4.0-155-generic
done
 Executing sudo update-grub
 Sourcing file `/etc/default/grub'
Sourcing file `/etc/default/grub.d/50-cloudimg-settings.cfg'
Sourcing file `/etc/default/grub.d/init-select.cfg'
Generating grub configuration file ...
Found linux image: /boot/vmlinuz-5.4.0-155-generic
Found initrd image: /boot/initrd.img-5.4.0-155-generic
done
 Done


Reboot the node (this sometimes generates an EOFError exception - ignore it and continue)

In [7]:
reboot = 'sudo reboot'

print(reboot)
node1.execute(reboot)

slice.wait_ssh(timeout=360,interval=10,progress=True)

print("Now testing SSH abilites to reconnect...",end="")
slice.update()
slice.test_ssh()
print("Reconnected!")

sudo reboot


Exception in thread Thread-313:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/paramiko/transport.py", line 2235, in run
    self.packetizer.close()
  File "/opt/conda/lib/python3.10/site-packages/paramiko/packet.py", line 207, in close
    self.__socket.close()
  File "/opt/conda/lib/python3.10/site-packages/paramiko/channel.py", line 669, in close
    self.transport._send_user_message(m)
  File "/opt/conda/lib/python3.10/site-packages/paramiko/transport.py", line 1937, in _send_user_message
    self._send_message(data)
  File "/opt/conda/lib/python3.10/site-packages/paramiko/transport.py", line 1913, in _send_message
    self.packetizer.send_message(data)
  File "/opt/conda/lib/python3.10/site-packages/paramiko/packet.py", line 425, in send_message
    self.write_all(out)
  File "/opt/conda/lib/python3.10/site-packages/paramiko/packet.py", line 361, in wr

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful
Now testing SSH abilites to reconnect...Reconnected!


Check that IOMMU was enabled

In [8]:
command = 'dmesg | grep -i IOMMU'

print('Observe that the modifications to boot configuration took place and IOMMU is detected')
stdout, stderr = node1.execute(command)

node1.config()

Observe that the modifications to boot configuration took place and IOMMU is detected
[    0.000000] Command line: BOOT_IMAGE=/boot/vmlinuz-5.4.0-155-generic root=LABEL=cloudimg-rootfs ro amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=8 console=tty1 console=ttyS0
[    0.033174] Kernel command line: BOOT_IMAGE=/boot/vmlinuz-5.4.0-155-generic root=LABEL=cloudimg-rootfs ro amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=8 console=tty1 console=ttyS0
[    1.417417] iommu: Default domain type: Passthrough (set via kernel command line)


'Done'

Disable IOMMU support in VFIO (the passing through doesn't actually work)

In [9]:
# Enable unsafe_noiommu_mode for the vfio module
command = "echo 1 | sudo tee /sys/module/vfio/parameters/enable_unsafe_noiommu_mode"

stdout, stderr = node1.execute(command)

1


## Install Docker compose

In [10]:
commands = ["sudo usermod -G docker ubuntu", 
            "mkdir -p ~/.docker/cli-plugins/",
            "curl -SL https://github.com/docker/compose/releases/download/v2.17.2/docker-compose-linux-x86_64 -o ~/.docker/cli-plugins/docker-compose",
            "chmod +x ~/.docker/cli-plugins/docker-compose",
            "curl -SL https://github.com/docker/buildx/releases/download/v0.11.2/buildx-v0.11.2.linux-amd64 -o ~/.docker/cli-plugins/docker-buildx",
            "chmod +x ~/.docker/cli-plugins/docker-buildx",
            "docker compose version",
            "docker container ps"
           ]

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node1.execute(command)
    
print('Done')

Executing sudo usermod -G docker ubuntu
Executing mkdir -p ~/.docker/cli-plugins/
Executing curl -SL https://github.com/docker/compose/releases/download/v2.17.2/docker-compose-linux-x86_64 -o ~/.docker/cli-plugins/docker-compose
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 51.9M  100 51.9M    0     0   108M      0 --:--:-- --:--:-- --:--:--  262M
 Executing chmod +x ~/.docker/cli-plugins/docker-compose
Executing curl -SL https://github.com/docker/buildx/releases/download/v0.11.2/buildx-v0.11.2.linux-amd64 -o ~/.docker/cli-plugins/docker-buildx
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 51.9M  100 

## Mount the storage

In [11]:
storage = node1.get_storage(storage_name)

stdout,stderr = node1.execute(f"sudo mkdir -p /mnt/{mount_point}; sudo chmod go+rw /mnt/{mount_point};"
                              f"sudo mount {storage.get_device_name()} /mnt/{mount_point}; "
                              f"df -h")

Filesystem      Size  Used Avail Use% Mounted on
udev            1.4G     0  1.4G   0% /dev
tmpfs           795M  1.3M  794M   1% /run
/dev/vda3        94G  3.5G   86G   4% /
tmpfs           3.9G     0  3.9G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs           3.9G     0  3.9G   0% /sys/fs/cgroup
/dev/vda1       549M  176K  549M   1% /boot/efi
/dev/loop0       64M   64M     0 100% /snap/core20/1974
/dev/loop2       54M   54M     0 100% /snap/snapd/19457
/dev/loop1       92M   92M     0 100% /snap/lxd/24061
tmpfs           795M     0  795M   0% /run/user/1000
/dev/vdb        984G  3.1G  931G   1% /mnt/xilinx-tools


## Program FPGA and run applications on it

First we checkout the `esnet-smartnic-fw` code, patch it and using an existing p4 artifact build a configuration of containers we can then execute.

In [12]:
#
# install existing dpdk and xilinx-labtools containers (pre-built) from
# https://github.com/esnet/smartnic-dpdk-docker and https://github.com/esnet/xilinx-labtools-docker
#
dpdk_docker = 'smartnic-dpdk-docker.tar.gz'
xilinx_labtools_docker = 'xilinx-labtools-docker-2023.1_0507_1903.tar.gz'
artifact = '/mnt/xilinx-tools/artifacts/msada/v0/artifacts.au280.p4_only.0.zip'

commands = [
    f"docker load < /mnt/{mount_point}/esnet-dockers/{dpdk_docker}",
    f"docker load < /mnt/{mount_point}/esnet-dockers/{xilinx_labtools_docker}",
    f"docker image ls"
]
for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node1.execute(command)

Executing docker load < /mnt/xilinx-tools/esnet-dockers/smartnic-dpdk-docker.tar.gz
Loaded image: smartnic-dpdk-docker:ubuntu-dev
Executing docker load < /mnt/xilinx-tools/esnet-dockers/xilinx-labtools-docker-2023.1_0507_1903.tar.gz
Loaded image: xilinx-labtools-docker:ubuntu-dev
Executing docker image ls
REPOSITORY               TAG          IMAGE ID       CREATED        SIZE
smartnic-dpdk-docker     ubuntu-dev   8448c2c791b5   2 months ago   696MB
xilinx-labtools-docker   ubuntu-dev   3fc9e77eb8e2   2 months ago   5.45GB


Next clone the repo and using externally provided p4 artifact build a container and a compose structure

In [13]:
# clone the esnet-smartnic-fw repo according to instructions https://github.com/esnet/esnet-smartnic-fw/tree/main (as of 09/2023)
# create a configuration environment file and build a container

# if the artifact file is called artifacts.au280.p4_only.0.zip then it translates into
# the following environment parameters
env_file = """
SN_HW_VER=0
SN_HW_BOARD=au280
SN_HW_APP_NAME=p4_only
"""

commands = [
    "git clone https://github.com/esnet/esnet-smartnic-fw.git",
    "cd ~/esnet-smartnic-fw; git submodule init; git submodule update",
    f"cp {artifact} ~/esnet-smartnic-fw/sn-hw/",
    f"echo '{env_file}' | sudo tee ~/esnet-smartnic-fw/.env",
]

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node1.execute(command)    

Executing git clone https://github.com/esnet/esnet-smartnic-fw.git
 Cloning into 'esnet-smartnic-fw'...
 Executing cd ~/esnet-smartnic-fw; git submodule init; git submodule update
Submodule path 'regio': checked out 'fbd8bdf1a7a628287e95c5cf667b14afc94203c5'
 Submodule 'regio' (https://github.com/esnet/regio) registered for path 'regio'
Cloning into '/home/ubuntu/esnet-smartnic-fw/regio'...
 Executing cp /mnt/xilinx-tools/artifacts/msada/v0/artifacts.au280.p4_only.0.zip ~/esnet-smartnic-fw/sn-hw/
Executing echo '
SN_HW_VER=0
SN_HW_BOARD=au280
SN_HW_APP_NAME=p4_only
' | sudo tee ~/esnet-smartnic-fw/.env

SN_HW_VER=0
SN_HW_BOARD=au280
SN_HW_APP_NAME=p4_only



Now build the artifacts.

In [14]:
# finally build, logging to a file
node_thread = node1.execute_thread("cd ~/esnet-smartnic-fw/; ./build.sh", output_file='esnet-smartnic-fw-docker.log')
stdout, stderr = node_thread.result()

### Test FPGA setup by accessing sn-cli under `smartnic-mgr-vfio-unlock` profile

We use the ESnet workflow to flash the FPGA and access `sn-cli` application to test whether CMACs are up

In [15]:
# set the FPGA device and the profile we want to execute
env_file = """
FPGA_PCIE_DEV=0000:1f:00
COMPOSE_PROFILES=smartnic-mgr-vfio-unlock
"""

# set execution profile to smartnic-mgr-vfio-unlock and run the stack
# notice we append to the pre-generated .env (it was generated as part of previous build)
commands = [
    f"echo '{env_file}' | tee -a ~/esnet-smartnic-fw/sn-stack/.env",
    "cd ~/esnet-smartnic-fw/sn-stack; docker compose up -d"
]

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node1.execute(command)  

Executing echo '
FPGA_PCIE_DEV=0000:1f:00
COMPOSE_PROFILES=smartnic-mgr-vfio-unlock
' | tee -a ~/esnet-smartnic-fw/sn-stack/.env

FPGA_PCIE_DEV=0000:1f:00
COMPOSE_PROFILES=smartnic-mgr-vfio-unlock

Executing cd ~/esnet-smartnic-fw/sn-stack; docker compose up -d
  Network sn-stack-ubuntu_default  Creating
 Network sn-stack-ubuntu_default  Created
 Volume "sn-stack-ubuntu_bitfiles"  Creating
 Volume "sn-stack-ubuntu_bitfiles"  Created
 Container sn-stack-ubuntu-smartnic-unpack-1  Creating
 Container sn-stack-ubuntu-xilinx-hwserver-1  Creating
 Container sn-stack-ubuntu-smartnic-unpack-1  Created
 Container sn-stack-ubuntu-xilinx-hwserver-1  Created
 Container sn-stack-ubuntu-smartnic-hw-1  Creating
 Container sn-stack-ubuntu-smartnic-hw-1  Created
 Container sn-stack-ubuntu-smartnic-fw-1  Creating
 Container sn-stack-ubuntu-smartnic-devbind-1  Creating
 Container sn-stack-ubuntu-smartnic-p4-1  Creating
 Container sn-stack-ubuntu-smartnic-devbind-1  Created
 Container sn-stack-ubuntu-smar

Run some healthchecks (should not see errors)

In [16]:
stdout, stderr = node1.execute("cd esnet-smartnic-fw/sn-stack/; docker container logs sn-stack-ubuntu-smartnic-devbind-1")


Network devices using DPDK-compatible driver
0000:1f:00.0 'Device 903f' drv=vfio-pci unused=
0000:1f:00.1 'Device 913f' drv=vfio-pci unused=

Network devices using kernel driver
0000:03:00.0 'Virtio network device 1041' if= drv=virtio-pci unused=vfio-pci 

No 'Baseband' devices detected

No 'Crypto' devices detected

No 'DMA' devices detected

No 'Eventdev' devices detected

No 'Mempool' devices detected

No 'Compress' devices detected

Misc (rawdev) devices using kernel driver
0000:04:00.0 'Virtio block device 1042' drv=virtio-pci unused=vfio-pci 
0000:08:00.0 'Virtio block device 1042' drv=virtio-pci unused=vfio-pci 

No 'Regex' devices detected
Driver:	vfio-pci
Driver:	vfio-pci
 + echo 1
+ sleep 1
+ dpdk-devbind.py -b vfio-pci 0000:1f:00.0 0000:1f:00.1
+ dpdk-devbind.py --status
+ lspci -D -kvm -s 0000:1f:00.0
+ grep '^Driver:	vfio-pci'
+ lspci -D -kvm -s 0000:1f:00.1
+ grep '^Driver:	vfio-pci'
+ touch /status/ok
+ sleep infinity
 

### Test sn-cli, configure CMACs

Should see normal looking output. If everything is 0x0000 or 0xffff, the binding to FPGA from VFIO did not work.

In [17]:
command = "cd esnet-smartnic-fw/sn-stack/; docker compose exec smartnic-fw sn-cli dev version"

stdout, stderr = node1.execute(command)

Device Version Info
	DNA:           0x400200000150c1260c606405
	USR_ACCESS:    0x64b9baac (1689893548)
	BUILD_STATUS:  0x07201752


Let's configure CMACs so we can test pktgen. It is important that at the end you see `MAC ENABLED/PHY UP -> UP` for both CMACs Rx and Tx. If not, it is possible FEC is not turned off in the dataplane switch.

In [18]:
# upload sn-cli config script
sn_cli_script = 'sn-cli-setup'

result = node1.upload_file(sn_cli_script, sn_cli_script)

commands = [
    f"chmod a+x {sn_cli_script}",
    f"mv {sn_cli_script} ~/esnet-smartnic-fw/sn-stack/scratch",
    f"cd ~/esnet-smartnic-fw/sn-stack/; docker compose exec smartnic-fw scratch/{sn_cli_script}"
]

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node1.execute(command)  

Executing chmod a+x sn-cli-setup
Executing mv sn-cli-setup ~/esnet-smartnic-fw/sn-stack/scratch
Executing cd ~/esnet-smartnic-fw/sn-stack/; docker compose exec smartnic-fw scratch/sn-cli-setup
Device Version Info
	DNA:           0x400200000150c1260c606405
	USR_ACCESS:    0x64b9baac (1689893548)
	BUILD_STATUS:  0x07201752
Applying mappings: 4
	[0]  CMAC0 -> CMAC0  OK
	[1]  CMAC1 -> CMAC1  OK
	[2]  HOST0 -> HOST0  OK
	[3]  HOST1 -> HOST1  OK
Applying mappings: 4
	[0]  CMAC0 -> CMAC0  OK
	[1]  CMAC1 -> CMAC1  OK
	[2]  HOST0 -> HOST0  OK
	[3]  HOST1 -> HOST1  OK
Applying mappings: 4
	[0]  CMAC0 -> CMAC0  OK
	[1]  CMAC1 -> CMAC1  OK
	[2]  HOST0 -> HOST0  OK
	[3]  HOST1 -> HOST1  OK
Applying mappings: 4
	[0]  CMAC0 -> HOST0  OK
	[1]  CMAC1 -> HOST1  OK
	[2]  HOST0 -> CMAC0  OK
	[3]  HOST1 -> CMAC1  OK
Applying mappings: 4
	[0]  CMAC0 -> BYPASS  OK
	[1]  CMAC1 -> BYPASS  OK
	[2]  HOST0 -> BYPASS  OK
	[3]  HOST1 -> BYPASS  OK
Ingress Switch Input Port Remap
	[0] CMAC0	CMAC0
	[1] CMAC1	CMAC1
	[

### Try pktgen

First we shutdown the docker stack, modify the profile to be `smartnic-mgr-dpdk-manual`, restart the stack. Then we access pktgen application and configure it to send some packets, finally we use the second host with CX-6 cards to snoop (tcpdump) and receive those packets.

In [19]:
# bring down the stack

command = "cd esnet-smartnic-fw/sn-stack/; docker compose down"

stdout, stderr = node1.execute(command)

  Container sn-stack-ubuntu-smartnic-fw-1  Stopping
 Container sn-stack-ubuntu-smartnic-fw-1  Stopping
 Container sn-stack-ubuntu-smartnic-p4-1  Stopping
 Container sn-stack-ubuntu-smartnic-vfio-unlock-1  Stopping
 Container sn-stack-ubuntu-smartnic-p4-1  Stopping
 Container sn-stack-ubuntu-smartnic-vfio-unlock-1  Stopping
 Container sn-stack-ubuntu-smartnic-fw-1  Stopped
 Container sn-stack-ubuntu-smartnic-fw-1  Removing
 Container sn-stack-ubuntu-smartnic-fw-1  Removed
 Container sn-stack-ubuntu-smartnic-p4-1  Stopped
 Container sn-stack-ubuntu-smartnic-p4-1  Removing
 Container sn-stack-ubuntu-smartnic-p4-1  Removed
 Container sn-stack-ubuntu-smartnic-vfio-unlock-1  Stopped
 Container sn-stack-ubuntu-smartnic-vfio-unlock-1  Removing
 Container sn-stack-ubuntu-smartnic-vfio-unlock-1  Removed
 Container sn-stack-ubuntu-smartnic-devbind-1  Stopping
 Container sn-stack-ubuntu-smartnic-devbind-1  Stopping
 Container sn-stack-ubuntu-smartnic-devbind-1  Stopped
 Container sn-stack-ubuntu-s

Modify the profile in the configuration .env file

In [20]:
# modify the profile to be `smartnic-mgr-dpdk-manual`
commands = [
    "sed -i 's/COMPOSE_PROFILES=smartnic-mgr-vfio-unlock/COMPOSE_PROFILES=smartnic-mgr-dpdk-manual/' ~/esnet-smartnic-fw/sn-stack/.env",
    "tail ~/esnet-smartnic-fw/sn-stack/.env"
]

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node1.execute(command)  

Executing sed -i 's/COMPOSE_PROFILES=smartnic-mgr-vfio-unlock/COMPOSE_PROFILES=smartnic-mgr-dpdk-manual/' ~/esnet-smartnic-fw/sn-stack/.env
Executing tail ~/esnet-smartnic-fw/sn-stack/.env

# block-start auto-generated by build
SMARTNIC_FW_IMAGE_URI=esnet-smartnic-fw:ubuntu-dev
LABTOOLS_IMAGE_URI=xilinx-labtools-docker:ubuntu-dev
SMARTNIC_DPDK_IMAGE_URI=smartnic-dpdk-docker:ubuntu-dev
# block-end   auto-generated by build

FPGA_PCIE_DEV=0000:1f:00
COMPOSE_PROFILES=smartnic-mgr-dpdk-manual



Restart the stack

In [21]:
command = "cd ~/esnet-smartnic-fw/sn-stack; docker compose up -d"
stdout, stderr = node1.execute(command)


  Network sn-stack-ubuntu_default  Creating
 Network sn-stack-ubuntu_default  Created
 Container sn-stack-ubuntu-xilinx-hwserver-1  Creating
 Container sn-stack-ubuntu-smartnic-unpack-1  Creating
 Container sn-stack-ubuntu-smartnic-unpack-1  Created
 Container sn-stack-ubuntu-xilinx-hwserver-1  Created
 Container sn-stack-ubuntu-smartnic-hw-1  Creating
 Container sn-stack-ubuntu-smartnic-hw-1  Created
 Container sn-stack-ubuntu-smartnic-devbind-1  Creating
 Container sn-stack-ubuntu-smartnic-fw-1  Creating
 Container sn-stack-ubuntu-smartnic-p4-1  Creating
 Container sn-stack-ubuntu-smartnic-fw-1  Created
 Container sn-stack-ubuntu-smartnic-devbind-1  Created
 Container sn-stack-ubuntu-smartnic-dpdk-1  Creating
 Container sn-stack-ubuntu-smartnic-p4-1  Created
 Container sn-stack-ubuntu-smartnic-dpdk-1  Created
 Container sn-stack-ubuntu-smartnic-unpack-1  Starting
 Container sn-stack-ubuntu-xilinx-hwserver-1  Starting
 Container sn-stack-ubuntu-smartnic-unpack-1  Started
 Container sn

The next step should be executed from the console. 
1. SSH into fpga-node
2. `cd ~/esnet-smartnic-fw/sn-stack`
3. `docker compose exec smartnic-dpdk bash`
4. `pktgen -a $SN_PCIE_DEV.0 -a $SN_PCIE_DEV.1 -l 3-7 -n 3 -d librte_net_qdma.so --file-prefix $SN_PCIE_DEV- -- -v -m [4:5].0 -m [6:7].1`

(pktgen should properly initialize, then at the pktgen prompt issue the following commands, which set unicast MAC addresses, set frame size to 128, 1% framerate, i.e. 1Gbps and start sending packets out of both ports)
```
set 0-1 dst mac 04:16:17:18:19:1a
set 0-1 src mac 14:16:17:18:19:10
set 0-1 size 128
set 0-1 rate 1
start 0
start 1
```
Next on cx-6-node enable the dataplane interfaces.

In [22]:
node2 = slice.get_node(name=cx6_node_name)

commands = [
    "sudo ip link set up enp7s0",
    "sudo ip link set up enp8s0"
]

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node2.execute(command)  

Executing sudo ip link set up enp7s0
Executing sudo ip link set up enp8s0


Try tcpdump on enp7s0, then on enp8s0

In [23]:
pktcount = 10
print("LISTENING ON enp7s0")
command = f"sudo tcpdump -nlvvxx -i enp7s0 -c {pktcount} tcp"

stdout, stderr = node2.execute(command)
print("LISTENING ON enp8s0")
command = f"sudo tcpdump -nlvvxx -i enp8s0 -c {pktcount} tcp"

stdout, stderr = node2.execute(command)

LISTENING ON enp7s0
21:42:57.803894 IP (tos 0x0, ttl 64, id 11194, offset 0, flags [none], proto TCP (6), length 110)
    192.168.0.1.1234 > 192.168.1.1.5678: Flags [.], cksum 0x3fa3 (correct), seq 74616:74686, ack 74640, win 8192, length 70
	0x0000:  0416 1718 191a 1416 1718 1910 0800 4500
 tcpdump: listening on enp7s0, link-type EN10MB (Ethernet), capture size 262144 bytes
 	0x0010:  006e 2bba 0000 4006 cc7d c0a8 0001 c0a8
	0x0020:  0101 04d2 162e 0001 2378 0001 2390 5010
	0x0030:  2000 3fa3 0000 7778 797a 3031 3233 3435
	0x0040:  6162 6364 6566 6768 696a 6b6c 6d6e 6f70
	0x0050:  7172 7374 7576 7778 797a 3031 3233 3435
	0x0060:  6162 6364 6566 6768 696a 6b6c 6d6e 6f70
	0x0070:  7172 7374 7576 7778 797a 3031
21:42:57.803894 IP (tos 0x0, ttl 64, id 11167, offset 0, flags [none], proto TCP (6), length 110)
    192.168.0.1.1234 > 192.168.1.1.5678: Flags [.], cksum 0x3fa3 (correct), seq 0:70, ack 1, win 8192, length 70
	0x0000:  0416 1718 191a 1416 1718 1910 0800 4500
	0x0010:  006e 2b9f 

Bring the stack down

In [24]:
command = "cd ~/esnet-smartnic-fw/sn-stack; docker compose down"

stdout, stderr = node1.execute(command)

  Container sn-stack-ubuntu-smartnic-p4-1  Stopping
 Container sn-stack-ubuntu-smartnic-p4-1  Stopping
 Container sn-stack-ubuntu-smartnic-fw-1  Stopping
 Container sn-stack-ubuntu-smartnic-fw-1  Stopping
 Container sn-stack-ubuntu-smartnic-dpdk-1  Stopping
 Container sn-stack-ubuntu-smartnic-dpdk-1  Stopping
 Container sn-stack-ubuntu-smartnic-p4-1  Stopped
 Container sn-stack-ubuntu-smartnic-p4-1  Removing
 Container sn-stack-ubuntu-smartnic-p4-1  Removed
 Container sn-stack-ubuntu-smartnic-fw-1  Stopped
 Container sn-stack-ubuntu-smartnic-fw-1  Removing
 Container sn-stack-ubuntu-smartnic-fw-1  Removed
 Container sn-stack-ubuntu-smartnic-dpdk-1  Stopped
 Container sn-stack-ubuntu-smartnic-dpdk-1  Removing
 Container sn-stack-ubuntu-smartnic-dpdk-1  Removed
 Container sn-stack-ubuntu-smartnic-devbind-1  Stopping
 Container sn-stack-ubuntu-smartnic-devbind-1  Stopping
 Container sn-stack-ubuntu-smartnic-devbind-1  Stopped
 Container sn-stack-ubuntu-smartnic-devbind-1  Removing
 Contai

## Extend the slice (as needed)

If you need to extend the storage slice, you can just execute the following two cells. They display the slice expiration date and optionally extend by 2 weeeks. 

In [ ]:
slice = fablib.get_slice(name=slice_name)
a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

Renew the slice

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice (as needed)

Please delete your slice when you are done with your experiment.


In [26]:
slice = fablib.get_slice(name=slice_name)
slice.delete()